In [10]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


In [4]:
load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print("Count of splits: ",len(splits))
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())



Count of splits:  66


NameError: name 'RunnableLambda' is not defined

In [11]:
# Retrieve and generate using the relevant snippets of the blog.
#retriever = vectorstore.as_retriever()
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=3)
prompt = hub.pull("rlm/rag-prompt")

In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
rag_chain =(
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
for chunk in rag_chain.stream("what is task decomposition?"):
    print(chunk,end="",flush=True)

Task decomposition is the process of breaking down a problem into multiple thought steps and generating multiple thoughts per step, creating a tree structure. It can be done using simple prompting, task-specific instructions, or human inputs. Task decomposition helps in transforming complex tasks into smaller and simpler steps for easier handling and understanding.